In [1]:
import gdspy
import numpy as np


In [2]:
# The GDSII file is called a library, which contains multiple cells.
lib = gdspy.GdsLibrary(unit=1e-6, precision=1e-9)

# Geometry must be placed in cells.
cell = lib.new_cell('hexagonal_cavity')

tol = 0.0001 # This control how good circles/arcs really is
fontsize = 10


In [3]:
# hexagonal waveguide inputs
t_waveguide = 0.4
w_waveguides = 0.7*np.ones(11)
r_waveguides = np.array([55, 55, 55, 55, 55, 55, 55, 75, 75, 75, 75])
r_coupling = np.array([80, 80, 80, 80, 80, 80, 80, 80, 80, 85, 85])
d_waveguides = np.array([17.65, 18.84, 20.03, 21.23, 22.42, 23.61, 24.81, 20.19, 21.38, 22.48, 23.68])
L_total = np.array([13500, 13500, 13500, 13500, 13500, 13500, 13500, 17000, 17000, 17000, 17000])
gap_N = np.array([0.4, 0.5, 0.6, 0.7, 0.8])
N = len(gap_N)
rotations_geometry = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])*np.pi/180
rotations_material = -rotations_geometry
#-------------

# aux inputs
l_square = 10.0
l_taper = 2800.0
w_taper = 0.30
#-------------

# doses and names
sample_number = "CCR_HexCav_002"
ebeam_brand = "EL7"
ebeam_energy = 150 # in keV
ebeam_dose = "4x200=800" # in uC/cm2
ebeam_current = 5 # in nA
field_size = 500
#-------------

# cells offsets
Nx_offset = 1000
Ny_offset = -1000
#-------------

# header offsets
dx_header = -58.5
dy_header = 700
#-------------

# aux functions
def waveguide_segment(waveguide, L, len_SC, layer):
    waveguide.segment(L, layer=layer)
    len_SC[0] = len_SC[0] + np.abs(L)
    
def waveguide_arc(waveguide, R, θi, θf, tol, len_SC, layer):
    waveguide.arc(R, θi, θf, tolerance=tol, layer=layer)
    len_SC[1] = len_SC[1] + R*np.abs(θi - θf)
#-------------


In [4]:
for k in np.arange(N):
    for i in np.arange(len(w_waveguides)):
        waveguides_layer = 0
        cavity_layer = 1
        header_layer = 200

        len_SC = np.array([0.0, 0.0])
        w = w_waveguides[i]
        R = r_waveguides[i]
        d = d_waveguides[i] + w
        π = np.pi
        θ_geo = rotations_geometry[i]
        L = L_total[i]
        gap = gap_N[k]
        x0 = i*Nx_offset - (k-N/2+1/2)*d*np.sign(np.sign(θ_geo)*2-1)
        y0 = k*Ny_offset

        waveguide_up = gdspy.Path(w, (0, 0))
        waveguide_arc(waveguide_up, R, -π, 0, tol, len_SC, layer=cavity_layer)
        j = 0
        while len_SC[0]+len_SC[1]<L_total[i]/2-2*np.pi*r_coupling[i]:
            L_j = j*2*d*np.sin(π/3)/(1+np.cos(π/3))
            L0 = d/np.sin(π/3) + L_j
            waveguide_segment(waveguide_up, L0, len_SC, layer=cavity_layer)
            waveguide_arc(waveguide_up, R, 0, π/3, tol, len_SC, layer=cavity_layer)
            L1 = R*(1+2*np.cos(π/3))/(2*np.sin(π/3)) + 2*L_j
            waveguide_segment(waveguide_up, L1, len_SC, layer=cavity_layer)
            waveguide_arc(waveguide_up, R, π/3, 2*π/3, tol, len_SC, layer=cavity_layer)
            L2 = -L1 + (d + R + 2*R*np.cos(π/3))/np.sin(π/3) + 4*L_j
            waveguide_segment(waveguide_up, L2, len_SC, layer=cavity_layer)
            waveguide_arc(waveguide_up, R, 2*π/3, π, tol, len_SC, layer=cavity_layer)
            L3 = L0 + L1*np.cos(π/3) - L2*np.cos(π/3)
            waveguide_segment(waveguide_up, L3, len_SC, layer=cavity_layer)
            L4 = L0 + d*np.sin(π/3)/(1+np.cos(π/3))
            waveguide_segment(waveguide_up, L4, len_SC, layer=cavity_layer)
            waveguide_arc(waveguide_up, R, π, 4*π/3, tol, len_SC, layer=cavity_layer)
            L5 = L1 + 2*d*np.sin(π/3)/(1+np.cos(π/3))
            waveguide_segment(waveguide_up, L5, len_SC, layer=cavity_layer)
            waveguide_arc(waveguide_up, R, 4*π/3, 5*π/3, tol, len_SC, layer=cavity_layer)
            L6 = L2 + 2*d*np.sin(π/3)/(1+np.cos(π/3))
            waveguide_segment(waveguide_up, L6, len_SC, layer=cavity_layer)
            waveguide_arc(waveguide_up, R, 5*π/3, 2*π, tol, len_SC, layer=cavity_layer)
            L7 = L3 + d*np.sin(π/3)/(1+np.cos(π/3))
            waveguide_segment(waveguide_up, L7, len_SC, layer=cavity_layer)
            j+=1
        
        j_max = j
        
        L_j = j*2*d*np.sin(π/3)/(1+np.cos(π/3))
        L0 = d/np.sin(π/3) + L_j
        waveguide_segment(waveguide_up, L0, len_SC, layer=cavity_layer)
        waveguide_arc(waveguide_up, R, 0, π/3, tol, len_SC, layer=cavity_layer)
        L1 = R*(1+2*np.cos(π/3))/(2*np.sin(π/3)) + 2*L_j
        waveguide_segment(waveguide_up, L1, len_SC, layer=cavity_layer)
        waveguide_arc(waveguide_up, R, π/3, 2*π/3, tol, len_SC, layer=cavity_layer)
        L2 = -L1 + (d + R + 2*R*np.cos(π/3))/np.sin(π/3) + 4*L_j
        waveguide_segment(waveguide_up, L2, len_SC, layer=cavity_layer)
        waveguide_arc(waveguide_up, R, 2*π/3, π, tol, len_SC, layer=cavity_layer)
        L3 = L0 + L1*np.cos(π/3) - L2*np.cos(π/3)
        waveguide_segment(waveguide_up, L3, len_SC, layer=cavity_layer)
        L4 = L0 + d*np.sin(π/3)/(1+np.cos(π/3))
        waveguide_segment(waveguide_up, L4, len_SC, layer=cavity_layer)
        y_up_ref_coupling = np.min(np.transpose(waveguide_up.polygons[-1])[1])

        # if θ_geo>0:
        #     waveguide_up.mirror((0,0), (0,1))
        #     waveguide_arc(waveguide_up, R, π, π-θ_geo, tol, len_SC, layer=waveguides_layer)
        # elif θ_geo<0:
        #     waveguide_arc(waveguide_up, R, 0, -θ_geo, tol, len_SC, layer=waveguides_layer)
        # waveguide_up.rotate(θ_geo)
        waveguide_up.translate(x0, y0)
        cell.add(waveguide_up)

        waveguide_down = gdspy.Path(w, (0, 0))
        waveguide_arc(waveguide_down, R, 0, π, tol, len_SC, layer=cavity_layer)
        for j in np.arange(j_max):
            L_j = j*2*d*np.sin(π/3)/(1+np.cos(π/3))
            L0 = d/np.sin(π/3) + L_j
            waveguide_segment(waveguide_down, L0, len_SC, layer=cavity_layer)
            waveguide_arc(waveguide_down, R, π, 4*π/3, tol, len_SC, layer=cavity_layer)
            L1 = R*(1+2*np.cos(π/3))/(2*np.sin(π/3)) + 2*L_j
            waveguide_segment(waveguide_down, L1, len_SC, layer=cavity_layer)
            waveguide_arc(waveguide_down, R, 4*π/3, 5*π/3, tol, len_SC, layer=cavity_layer)
            L2 = -L1 + (d + R + 2*R*np.cos(π/3))/np.sin(π/3) + 4*L_j
            waveguide_segment(waveguide_down, L2, len_SC, layer=cavity_layer)
            waveguide_arc(waveguide_down, R, 5*π/3, 2*π, tol, len_SC, layer=cavity_layer)
            L3 = L0 + L1*np.cos(π/3) - L2*np.cos(π/3)
            waveguide_segment(waveguide_down, L3, len_SC, layer=cavity_layer)
            L4 = L0 + d*np.sin(π/3)/(1+np.cos(π/3))
            waveguide_segment(waveguide_down, L4, len_SC, layer=cavity_layer)
            waveguide_arc(waveguide_down, R, 0, π/3, tol, len_SC, layer=cavity_layer)
            L5 = L1 + 2*d*np.sin(π/3)/(1+np.cos(π/3))
            waveguide_segment(waveguide_down, L5, len_SC, layer=cavity_layer)
            waveguide_arc(waveguide_down, R, π/3, 2*π/3, tol, len_SC, layer=cavity_layer)
            L6 = L2 + 2*d*np.sin(π/3)/(1+np.cos(π/3))
            waveguide_segment(waveguide_down, L6, len_SC, layer=cavity_layer)
            waveguide_arc(waveguide_down, R, 2*π/3, π, tol, len_SC, layer=cavity_layer)
            L7 = L3 + d*np.sin(π/3)/(1+np.cos(π/3))
            waveguide_segment(waveguide_down, L7, len_SC, layer=cavity_layer)
            
        waveguide_segment(waveguide_down, L0+2*d*np.sin(π/3)/(1+np.cos(π/3)), len_SC, layer=cavity_layer)
        waveguide_arc(waveguide_down, R, π, 4*π/3, tol, len_SC, layer=cavity_layer)
        waveguide_segment(waveguide_down, (r_coupling[i]*(1+np.cos(π/3))-d-R*(1-np.cos(π/3)))/np.cos(π/6), len_SC, layer=cavity_layer)
        waveguide_arc(waveguide_down, r_coupling[i], π/3, -π/2, tol, len_SC, layer=cavity_layer)
        x0wav = np.min(np.transpose(waveguide_down.polygons[-1])[0])
        y0wav = np.min(np.transpose(waveguide_down.polygons[-1])[1]) + w/2
        waveguide_arc(waveguide_down, r_coupling[i], -π/2, -π, tol, len_SC, layer=cavity_layer)
        y_down_ref_coupling = np.max(np.transpose(waveguide_down.polygons[-1])[1])
        waveguide_segment(waveguide_down, abs(y_up_ref_coupling-y_down_ref_coupling), len_SC, layer=cavity_layer)

        # if θ_geo>0:
        #     waveguide_down.mirror((0,0), (0,1))
        #     waveguide_arc(waveguide_down, R, 0, -θ_geo, tol, len_SC, layer=waveguides_layer)
        # elif θ_geo<0:
        #     waveguide_arc(waveguide_down, R, π, π-θ_geo, tol, len_SC, layer=waveguides_layer)
        # waveguide_down.rotate(θ_geo)
        waveguide_down.translate(x0, y0)
        cell.add(waveguide_down)
        
        # waveguides now
        waveguide = gdspy.Path(w, (x0+x0wav, y0+y0wav-w-gap))
        waveguide.segment(1000*int((abs(x0+x0wav)+500)/1000)+500-(x0+x0wav) - R - (20+w)*(N-k), "+x", layer=waveguides_layer)
        waveguide.arc(R, -π/2, 0, tolerance=tol, layer=waveguides_layer)
        y_up_ref = np.max(np.transpose(waveguide.polygons[-1])[1])
        waveguide.segment(abs(y_up_ref-500), layer=waveguides_layer)
        waveguide.segment(l_taper, final_width=w_taper, layer=waveguides_layer)
        cell.add(waveguide)
        
        x_square_up, y_square_up = np.mean(np.transpose(waveguide.polygons[-1])[0]), np.max(np.transpose(waveguide.polygons[-1])[1])
        square = gdspy.Rectangle((0, 0), (l_square, l_square), layer=waveguides_layer)
        square.translate(x_square_up-l_square/2, y_square_up-l_square)
        cell.add(square)
        
        waveguide = gdspy.Path(w, (x0+x0wav, y0+y0wav-w-gap))
        waveguide.segment(-1000*int((abs(x0+x0wav)+500)/1000)+500+(x0+x0wav) - R - (20+w)*(k+1), "-x", layer=waveguides_layer)
        waveguide.arc(R, π/2, π, tolerance=tol, layer=waveguides_layer)
        y_down_ref = np.min(np.transpose(waveguide.polygons[-1])[1])
        waveguide.segment(abs(y_down_ref+5500), layer=waveguides_layer)
        waveguide.segment(l_taper, final_width=w_taper, layer=waveguides_layer)
        cell.add(waveguide)
        
        x_square_down, y_square_down = np.mean(np.transpose(waveguide.polygons[-1])[0]), np.min(np.transpose(waveguide.polygons[-1])[1])
        square = gdspy.Rectangle((0, 0), (l_square, l_square), layer=waveguides_layer)
        square.translate(x_square_down-l_square/2, y_square_down)
        cell.add(square)
        
        # header
        
        if k==0:
            header1 = "angle:"+str(round(rotations_material[i]*180/π, 3))+"\n"
            header2 = "w:"+str(round(w, 3))+"\n"
            header3 = "R:"+str(round(R, 3))+"\n"
            header4 = "R_in:"+str(round(r_coupling[i], 3))+"\n"
            header5 = "d:"+str(round(d_waveguides[i], 3))+"\n"
            header6 = "SL:"+str(round(len_SC[0], 3))+"\n"
            header7 = "CL:"+str(round(len_SC[1], 3))+"\n"
            header8 = "L:"+str(round(len_SC[0]+len_SC[1], 3))+"\n"
            header = gdspy.Text(header1+header2+header3+header4+header5+header6+header7+header8, fontsize, layer=header_layer)
            header.translate(x0+dx_header, y0+dy_header)
            cell.add(header)
            if i==0:
                header1 = sample_number+"\n"
                header2 = "Brand:"+ebeam_brand+"\n"
                header3 = "Energy:"+str(ebeam_energy)+"keV \n"
                header4 = "Current:"+str(ebeam_current)+"nA \n"
                header5 = "Dose:"+str(ebeam_dose)+"uC/cm2 \n\n"
                header6 = "t:"+str(round(t_waveguide, 3))+"\n"
                header7 = "w_taper:"+str(round(w_taper, 3))
                header_all = header1+header2+header3+header4+header5+header6+header7
                header = gdspy.Text(header_all, fontsize, layer=header_layer)
                header.rotate(π/2)
                header.translate(x0+dx_header-175, y0+dy_header-70)
                cell.add(header)

                # axes
                material_axis = gdspy.Path(3, (x0+dx_header, y0+dy_header+100))
                material_axis.segment(50, "-y", layer=header_layer)
                material_axis.arc(3, -π, -π/2, layer=header_layer)
                material_axis.segment(50, "+x", layer=header_layer)
                cell.add(material_axis)

                material_X_label = gdspy.Text("X", fontsize, (x0+dx_header+50, y0+dy_header+50), layer=header_layer)
                cell.add(material_X_label)

                material_Y_label = gdspy.Text("Y", fontsize, (x0+dx_header, y0+dy_header+100), layer=header_layer)
                cell.add(material_Y_label)

                brand = gdspy.Text("Z-cut BSBS", fontsize, (x0+dx_header+50, y0+dy_header+100), layer=header_layer)
                cell.add(brand)
                
                header_up = gdspy.Text(header_all, fontsize, layer=header_layer)
                header_up.translate(x0+dx_header+6920-80, y0+dy_header-70+l_taper+300-550)
                cell.add(header_up)
                
                header_down = gdspy.Text(header_all, fontsize, layer=header_layer)
                header_down.translate(x0+dx_header+7000-750, y0+dy_header-70-3*l_taper-380+40)
                cell.add(header_down)
                
    header_gap = gdspy.Text("gap:"+str(round(gap, 3))+"\n", fontsize, layer=header_layer)
    header_gap.rotate(π/2)
    header_gap.translate(x0+dx_header-10525+230, y0+dy_header+300-1000+350)
    cell.add(header_gap)
    

In [5]:
# waveguide/microring inputs
w_waveguides = np.array([0.7, 1.4])
r_waveguides = np.array([30.0, 90.0])
gaps = np.linspace(0.35, 1.05, 15)
r_bend = 100.0
l_coupling = 400.0
#-------------

# offsets
dx_global = 6000
dy_global = 250-333

dx_i = 25.0
dy_i = -333
#-------------

# header offsets
dx_header = -110
dy_header = 20
#-------------


In [6]:
# for j in np.arange(len(r_waveguides)):
#     for i in np.arange(len(gaps)):
#         waveguides_layer = len(L_total)
#         rings_layer = len(L_total)+1
#         header_layer = 200
        
#         w = w_waveguides[j]
#         R = r_waveguides[j]
#         G = gaps[i]

#         x0 = (i-len(gaps)/2+1/2)*dx_i + j*Nx_offset
#         y0 = i*dy_i
        
#         ring = gdspy.Path(w, (x0, y0))
#         ring.arc(R, -np.pi/2, 3*np.pi/2, tolerance=tol, layer=rings_layer)
#         ring.translate(dx_global, dy_global)
#         cell.add(ring)
        
#         header1 = "w:"+str(w)+"\n"
#         header2 = "G:"+str(round(G, 3))+"\n"
#         header3 = "R:"+str(R)+"\n"
#         header = gdspy.Text(header1+header2+header3, fontsize, layer=header_layer)
#         header.translate(x0-R+dx_header+dx_global, y0+R+dy_header+dy_global)
#         cell.add(header)
        
#         waveguide = gdspy.Path(w, (x0, y0-w-G))
#         waveguide.segment(l_coupling/2, "+x", layer=waveguides_layer)
#         waveguide.arc(r_bend, -np.pi/2, 0, tolerance=tol, layer=waveguides_layer)
#         waveguide.translate(dx_global, dy_global)
#         y_ref = np.max(np.transpose(waveguide.polygons[-1])[1])
#         waveguide.segment(abs(500-y_ref), layer=waveguides_layer)
#         waveguide.segment(l_taper, final_width=w_taper, layer=waveguides_layer)
#         cell.add(waveguide)
        
#         x_square = (np.max(np.transpose(waveguide.polygons[-1])[0])+np.min(np.transpose(waveguide.polygons[-1])[0]))/2 - l_square/2
#         y_square = np.max(np.transpose(waveguide.polygons[-1])[1]) - l_square
#         square = gdspy.Rectangle((0, 0), (l_square, l_square), layer=waveguides_layer)
#         square.translate(x_square, y_square)
#         cell.add(square)

#         waveguide = gdspy.Path(w, (x0, y0-w-G))
#         waveguide.segment(l_coupling/2, "-x", layer=waveguides_layer)
#         waveguide.arc(r_bend, np.pi/2, np.pi, tolerance=tol, layer=waveguides_layer)
#         waveguide.translate(dx_global, dy_global)
#         y_ref = np.min(np.transpose(waveguide.polygons[-1])[1])
#         waveguide.segment(abs(y_ref+5500), layer=waveguides_layer)
#         waveguide.segment(l_taper, final_width=w_taper, layer=waveguides_layer)
#         cell.add(waveguide)
        
#         x_square = (np.max(np.transpose(waveguide.polygons[-1])[0])+np.min(np.transpose(waveguide.polygons[-1])[0]))/2 - l_square/2
#         y_square = np.min(np.transpose(waveguide.polygons[-1])[1])
#         square = gdspy.Rectangle((0, 0), (l_square, l_square), layer=waveguides_layer)
#         square.translate(x_square, y_square)
#         cell.add(square)


In [7]:
# offsets markers
dx_global = 0
dy_global = -8000

dx_markers = 5000
dy_markers = 11000
#-------------

flag = 1
for i in range(2):
    for j in range(2):
        for k in range(4):
            cross = gdspy.Rectangle((-0.5, -0.5), (99.5, 0.5), layer=header_layer)
            aux = k*np.pi/2
            cross.rotate(aux)
            cross.translate(1.5*np.cos(aux), 1.5*np.sin(aux))
            cross.translate(dx_global+i*dx_markers, dy_global+j*dy_markers)
            cell.add(cross)

        small_marker_1 = gdspy.Rectangle((0, 0), (0.5, 0.5), layer=header_layer)
        small_marker_2 = gdspy.Rectangle((0, 0), (-0.5, -0.5), layer=header_layer)
        small_marker_1.translate(dx_global+i*dx_markers, dy_global+j*dy_markers)
        small_marker_2.translate(dx_global+i*dx_markers, dy_global+j*dy_markers)
        cell.add(small_marker_1)
        cell.add(small_marker_2)

        circle_markes_1 = gdspy.Round((0, 0), 2.5, inner_radius=2.0, tolerance=tol, layer=header_layer)
        circle_markes_2 = gdspy.Round((0, 0), 3.5, inner_radius=3.0, tolerance=tol, layer=header_layer)
        circle_markes_1.translate(dx_global+i*dx_markers, dy_global+j*dy_markers)
        circle_markes_2.translate(dx_global+i*dx_markers, dy_global+j*dy_markers)
        cell.add(circle_markes_1)
        cell.add(circle_markes_2)
        
        marker_header = gdspy.Text(str(flag), 3*fontsize, layer=header_layer)
        marker_header.translate(50, 50)
        marker_header.translate(dx_global+i*dx_markers, dy_global+j*dy_markers)
        cell.add(marker_header)
        flag = flag+1
    

In [8]:
for i in np.arange(21):
    for j in np.arange(23):
        square = gdspy.Rectangle((0, 0), (1000,1000), layer=255)
        square.translate(i*field_size-500, j*field_size-4500-1000*(N-1))
        cell.add(square)
        

In [9]:
# Save the library in a file called 'first.gds'.
lib.write_gds(sample_number+'.gds')
